In [1]:
import requests
import pandas as pd
import glob

In [8]:
# récupérer tous les fichiers du dossier videos et dédupliquer les lignes
files = glob.glob("market_sport_fr.csv")

df = []
for f in files:
    csv = pd.read_csv(f)
    df.append(csv)
df = pd.concat(df,ignore_index=True)
df.drop_duplicates(subset=['video_id'], keep="first", inplace=True)
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 997 entries, 0 to 996
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   video_id      997 non-null    str  
 1   title         997 non-null    str  
 2   description   593 non-null    str  
 3   channel       997 non-null    str  
 4   published_at  997 non-null    str  
 5   duration      997 non-null    str  
 6   views         997 non-null    int64
 7   likes         997 non-null    int64
 8   comments      997 non-null    int64
 9   channel_id    997 non-null    str  
 10  category_id   997 non-null    int64
 11  language      985 non-null    str  
dtypes: int64(4), str(8)
memory usage: 93.6 KB


In [9]:
# langues: supprimer les lignes avec valeurs
df.dropna(subset="language",axis=0,inplace=True)

# langues: regrouper fr_fr, be_fr sous 'fr', pareil pour 'en'
df.loc[df['language'].str.contains('^fr'),'language'] = "fr"
df.loc[df['language'].str.contains('^en'),'language'] = "en"

In [10]:
#supprimer autres langues
df['language'].value_counts()

language
fr         427
en         392
hi         117
ru           6
th           4
zxx          4
pt-PT        4
de           4
es           4
bn           3
it           3
ur           2
id           2
pt-BR        2
te           2
zh           1
ta           1
de-AT        1
pa           1
bn-IN        1
ko           1
el           1
und          1
hi-Latn      1
Name: count, dtype: int64

In [11]:
# calculer le taux d'engagement pour voir en un coup d'oeil les vidéos ayant reçu le plus d'intéractions
df["Engagement total"] = df["likes"] + df["comments"]
df["Taux d'engagement (%)"] = (df["Engagement total"] / df["views"].replace(0, 1)) * 100

In [12]:
# transofrmer les durées (ici min/sec) en secondes (plus simple pour l'analyse
import re

# 1. Conversion en timedelta (errors='coerce' transforme les formats invalides en NaT)
df['duration_td'] = pd.to_timedelta(df['duration'], errors='coerce')

# 2. Conversion en secondes
# .astype('Int64') permet de garder les NaN si nécessaire sans faire planter le code
df['Durée (s)'] = df['duration_td'].dt.total_seconds().astype('Int64')

# OPTIONNEL : Si vous voulez remplacer les vides par 0 et avoir des entiers classiques
# df['Durée (s)'] = df['duration_td'].dt.total_seconds().fillna(0).astype(int)

print(df[['duration', 'Durée (s)']])

     duration  Durée (s)
0      PT1M1S         61
1       PT10S         10
2       PT15S         15
3       PT13S         13
4       PT29S         29
..        ...        ...
992     PT43S         43
993   PT9M24S        564
994     PT25S         25
995  PT38M44S       2324
996      PT8S          8

[985 rows x 2 columns]


In [13]:
# colonnes avec valeurs nulls
df.isna().sum()

video_id                   0
title                      0
description              403
channel                    0
published_at               0
duration                   0
views                      0
likes                      0
comments                   0
channel_id                 0
category_id                0
language                   0
Engagement total           0
Taux d'engagement (%)      0
duration_td                0
Durée (s)                  0
dtype: int64

In [16]:
df.to_csv("sport_clean.csv", index=False)
